In [39]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.stats import entropy

## Question 1

In [27]:
names = ['sex', 'length', 'diameter', 'height', 'whole_weight',
         'shucked_weight', 'viscera_weight', 'shell_weight', 'rings']
data = pd.read_csv("./data/abalone.csv", header=None, names=names)
data.head()

y0 = data['rings'].values
bins = np.array([0, 8.5, 10.5, 30])
y = np.digitize(y0, bins=bins)
y

X = data.loc[:, :'shell_weight']
X['sex'] = X['sex'].astype('category').cat.codes
X = X.values
X

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


array([3, 1, 2, ..., 2, 2, 3])

array([[2.    , 0.455 , 0.365 , ..., 0.2245, 0.101 , 0.15  ],
       [2.    , 0.35  , 0.265 , ..., 0.0995, 0.0485, 0.07  ],
       [0.    , 0.53  , 0.42  , ..., 0.2565, 0.1415, 0.21  ],
       ...,
       [2.    , 0.6   , 0.475 , ..., 0.5255, 0.2875, 0.308 ],
       [0.    , 0.625 , 0.485 , ..., 0.531 , 0.261 , 0.296 ],
       [2.    , 0.71  , 0.555 , ..., 0.9455, 0.3765, 0.495 ]])

In [54]:
model = KMeans(n_clusters=3)
model.fit(X)
km_centroids = model.cluster_centers_

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [48]:
def single_class_entropy(y):
    _, counts = np.unique(y, return_counts=True)
    prop = counts / len(y)
    return entropy(prop)

def single_class_purity(y):
    _, counts = np.unique(y, return_counts=True)
    prop = counts / len(y)
    return np.max(prop)


def evaluate(y_predicted, clusters):
    
    entropy = 0
    purity = 0
    
    for cluster in np.unique(clusters):
        
        class_distr = y_predicted[clusters == cluster]
        
        cluster_weight = len(class_distr) / len(clusters)
        
        entropy += single_class_entropy(class_distr) * cluster_weight
        purity += single_class_purity(class_distr) * cluster_weight
    
    return entropy, purity

evaluate(model.predict(X), y)

(0.9440174748968634, 0.524778549197989)

## Question 2

### a)
What is expecation maximisation?

It is an iterative methdo for estimating the parameters in a statistical model. It consists of three steps:
    (1) Initialization -- initialize the parameters of the model randomly (or according to some other method)
    (2) Expectation -- define a likelihood prediction function over the data using the current parameters.
    (3) Maximisation -- update the model to maximize the likelihood function over the data
(2) and (3) are repeated until convergence.

For example, in Gaussian Mixture Models:

Consider data to have been generated by one of N Gaussian distributions. We want to know which distribution each instance belongs.
The parameters are MEAN and SD of each of the Gaussians
steps:
    (1) Initialisation -- either initialize parameters randomly, or estimate (e.g. with KMeans)
    (2) Exepctation -- define a likelihood using current estimates for MEAN and SD
    (3) Maximisation -- assign each instance to a class that gives it the highest likelihood. Then re-estimate the parameters.

### b)
Effect of initializing using K-Means

If KMeans does a good job of approximating the true clusters: GMM should converge quickly and on a good approximation of true clusters.
If KMeans does a terrible job of approximating the true clusters: GMM may take longer to converge and/or find a local solution.

KMeans is basically the same as GMM, except GMM is probabilistic.

In [61]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=3)
gmm.fit(X)

gmm_centroids = gmm.means_

clusters = gmm.predict(X)

evaluate(clusters, y)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=3, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

(0.9599168786955169, 0.5149628920277711)

GMM is using KMM to come up with its initial clusters centroids, thus the two methods give very similar results.

In [62]:
km_centroids
gmm_centroids

array([[ 1.99098474e+00,  5.79323856e-01,  4.53959778e-01,
         1.56553398e-01,  1.05317372e+00,  4.60087032e-01,
         2.28861304e-01,  2.99109570e-01],
       [-2.64233080e-14,  5.79326187e-01,  4.54931087e-01,
         1.58078101e-01,  1.04727221e+00,  4.46505743e-01,
         2.30848775e-01,  3.02222052e-01],
       [ 1.06857943e+00,  4.17585724e-01,  3.18383485e-01,
         1.05360392e-01,  4.02548635e-01,  1.78093772e-01,
         8.59573128e-02,  1.20068929e-01]])

array([[1.        , 0.4277459 , 0.32649404, 0.10799553, 0.43136252,
        0.19103502, 0.09201006, 0.12818219],
       [2.        , 0.56139071, 0.43928665, 0.15138089, 0.99145942,
        0.43294601, 0.2155445 , 0.28196924],
       [0.        , 0.57909334, 0.45473221, 0.15801071, 1.04653213,
        0.44618783, 0.2306886 , 0.30200995]])